# Libs

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import scipy.stats
from scipy.stats import invgauss
from scipy.stats import chi2
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')


DIR_DATA = os.getcwd() + '/data/'

# Import data

In [ ]:
timestamp = "Timestamp"

df_dataset = pd.read_csv(DIR_DATA + 'Digestor.csv', sep=';', decimal='.')

df_dataset[timestamp]= pd.to_datetime(df_dataset[timestamp],format="%Y-%m-%d %H:%M:%S")
print(df_dataset[timestamp].min())
print(df_dataset[timestamp].max())

for val in df_dataset:
    if val != timestamp:
        df_dataset[val] = pd.to_numeric(df_dataset[val],errors = "coerce")
df_dataset = df_dataset.dropna()

print(df_dataset.shape)

cols = list(df_dataset.columns)
cols.remove(timestamp)

df_dataset.head()

# Remove offs

In [ ]:
def drop_transitorio_desligado(
    dataset,
    variavel,
    limite,
    intervalo,
    timestamp,
    pre_corte=0,
    pos_corte=0,
    pp_residual=0,
):
    final = False
    df_aux = dataset.loc[:, [variavel, timestamp]].copy()
    df_aux["status"] = 1  # Ligado
    ############################   Descida  ########################################
    ## Se uma amostra está abaixo do limite e permance nesse nivel por
    ## por um intervalo, o status desse periodo é setado como 0, o que indicada
    ## que o motor está desligado
    ###############################################################################
    data_aux = df_aux[timestamp].min()  # Primeira data do dataframe
    list_periods = []
    while True:
        # Pega a data da primeira amostra com o valor abaixo do limite
        df_amostra = df_aux[
            (df_aux[variavel] <= limite) & (df_aux[timestamp] >= data_aux)
        ]

        if not df_amostra.empty:
            data_min = df_amostra[timestamp].min()
        else:
            break

        # Pega a primeira data da amostra acima do valor limite depois da amostra acima
        df_amostra = df_aux[
            (df_aux[variavel] > limite) & (df_aux[timestamp] > data_min)
        ]

        if not df_amostra.empty:
            data_aux = df_amostra[timestamp].min()
        else:
            data_aux = df_aux[timestamp].max()
            final = True

        # Tira a diferença entre as duas amostras,
        dif_date = (data_aux - data_min).total_seconds()
        dif_date = dif_date/60

        # Caso o valor da diferença seja maior que >= 3600s (1 hora) o motor está desligado
        if dif_date >= intervalo:
            list_periods.append(
                {"date_ini": data_min, "date_end": data_aux, "type": "desligado"}
            )
            mask = (df_aux[timestamp] >= data_min) & (df_aux[timestamp] <= data_aux)
            df_aux["status"].loc[mask] = 0  # Status Desligado

            if pre_corte != 0:
                date_ini_pre_corte = data_min - timedelta(minutes=pre_corte)
                date_end_pre_corte = data_min

                mask_pre_corte = (df_aux[timestamp] >= date_ini_pre_corte) & (
                    df_aux[timestamp] <= date_end_pre_corte
                )
                df_aux["status"].loc[mask_pre_corte] = 0  # Status Desligado
                list_periods.append(
                    {
                        "date_ini": date_ini_pre_corte,
                        "date_end": date_end_pre_corte,
                        "type": "transitorio",
                    }
                )

            if pos_corte != 0:
                date_ini_pos_corte = data_aux
                date_end_pos_corte = data_aux + timedelta(minutes=pos_corte)

                mask_pos_corte = (df_aux[timestamp] >= date_ini_pos_corte) & (
                    df_aux[timestamp] <= date_end_pos_corte
                )
                df_aux["status"].loc[mask_pos_corte] = 0  # Status Desligado
                list_periods.append(
                    {
                        "date_ini": date_ini_pos_corte,
                        "date_end": date_end_pos_corte,
                        "type": "transitorio",
                    }
                )

        if final:
            break

    ############################   Subida  ########################################
    ## Essa parte do código serve para pegar picos onde o motor volta a "funcionar"
    ## por menos do intervalo, ou seja, ele estava desligado, deu um pique de menos
    ## de uma hora e voltou a ficar desligado
    ###############################################################################
    data_aux = df_aux[timestamp].min()
    while True:
        # Pega a primeira data da amostra com status ligado
        df_amostra = df_aux[(df_aux["status"] == 1) & (df_aux[timestamp] >= data_aux)]

        if not df_amostra.empty:
            data_min = df_amostra[timestamp].min()
        else:
            break

        # Pega a amostra com o status deligado após a data acima
        df_amostra = df_aux[(df_aux["status"] == 0) & (df_aux[timestamp] > data_min)]

        if not df_amostra.empty:
            data_aux = df_amostra[timestamp].min()
        else:
            break

        # Tira a diferença entre as duas amostras,
        dif_date = (data_aux - data_min).total_seconds()

        # Caso o valor da diferença seja maior que < 3600s (1 hora) o motor está desligado
        if dif_date < intervalo:
            list_periods.append(
                {"date_ini": data_min, "date_end": data_aux, "type": "desligado"}
            )
            mask = (df_aux[timestamp] >= data_min) & (df_aux[timestamp] <= data_aux)
            df_aux["status"].loc[mask] = 0

    df_aux["status"].iloc[:pp_residual] = 0
    df_return = dataset.copy()
    df_return.drop(df_aux[df_aux["status"] == 0].index, inplace=True)
    return df_return, df_aux, list_periods

pre_process = []
pp_var_ref_desligado = "211S001M.TT"
pp_valor_ref_desligado = 5
pp_tempo_ref_desligado = 0
pp_pre_corte_transitorio = 60
pp_pos_corte_transitorio = 60
pre_process.append(  
{
   "after_cut": pp_pos_corte_transitorio,
   "interval_off": pp_tempo_ref_desligado,
   "limit_off": pp_valor_ref_desligado,
   "pre_cut": pp_pre_corte_transitorio,
   "variable_off": pp_var_ref_desligado
  })

for pro in pre_process:
    df_dataset_ppd,_,_ = drop_transitorio_desligado(df_dataset,pro["variable_off"],pro["limit_off"],pro["interval_off"],timestamp,pre_corte=pro["pre_cut"],pos_corte=pro["after_cut"])

df_dataset_ppd

# Select training periods

In [ ]:
# === Pré-processamento ===
df = df_train.copy()
time = df[timestamp].to_list()
df = df.drop(timestamp, axis=1)

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# === Definindo Autoencoder ===
input_dim = df_scaled.shape[1]
encoding_dim = min(20, input_dim // 2)  # Ajustável

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='linear')(encoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# === Treinamento do Autoencoder ===
autoencoder.fit(df_scaled, df_scaled,
                epochs=10,
                batch_size=32,
                shuffle=True,
                verbose=1)

# === Reconstrução e erro ===
reconstructed = autoencoder.predict(df_scaled)
reconstruction_error = np.mean((df_scaled - reconstructed) ** 2, axis=1)

# === Cálculo do erro com suavização ===
df_error = pd.DataFrame(reconstruction_error, columns=["error-mse"])
df_error['error-ewm'] = df_error['error-mse'].ewm(alpha=0.01).mean()

# === Plotando ===
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=df_error['error-ewm'], mode="lines", name="Erro Reconstrução EWM"))
fig.update_layout(title="Erro de Reconstrução com Autoencoder", xaxis_title="Tempo", yaxis_title="Erro")
fig.show()

# Train autoencoder

In [ ]:
def train_stable_autoencoder(X_stable, encoding_dim=8, epochs=100, batch_size=32, verbose=1):
    input_dim = X_stable.shape[1]

    # Definição do modelo
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim * 2, activation='relu')(input_layer)
    encoded = Dense(encoding_dim, activation='relu')(encoded)

    decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
    decoded = Dense(input_dim, activation='linear')(decoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    # Treinamento
    autoencoder.fit(X_stable, X_stable,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    verbose=verbose)

    # Modelo encoder para projeção no espaço latente
    encoder = Model(inputs=input_layer, outputs=encoded)

    return autoencoder, encoder

start_date_train = pd.to_datetime('2023-09-02 00:00:00')
end_date_train = pd.to_datetime('2023-09-12 00:00:00')

mask = (df_dataset[timestamp] >= start_date_train) & (df_dataset[timestamp] <= end_date_train)
df_train = df_dataset.loc[mask].drop(columns=[timestamp])

# Escala os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(df_train)

# Treina o autoencoder
autoencoder, encoder = train_stable_autoencoder(X_train, epochs=10, encoding_dim=12)

# Predict

In [ ]:
start_date_test = pd.to_datetime('2023-09-01 00:00:00')
end_date_test = pd.to_datetime('2025-09-12 00:00:00')

mask = (df_dataset[timestamp] >= start_date_test) & (df_dataset[timestamp] <= end_date_test)
df_test = df_dataset.loc[mask]
eixo_teste = df_test[timestamp].to_list()
df_test = df_test.drop(columns=[timestamp])

# Pré-processar novos dados com o mesmo scaler usado nos dados de treino
X_test = scaler.transform(df_test)

# Projeção no espaço latente
X_latent = encoder.predict(X_test)

# Reconstrução
X_reconstructed = autoencoder.predict(X_test)


# Calculate reconstruction error

In [ ]:
reconstruction_error = np.mean((X_test - X_reconstructed)**2, axis=1)

df_result = pd.DataFrame({
    "timestamp": eixo_teste,
    "reconstruction_error": reconstruction_error
})

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=eixo_teste, y=reconstruction_error, mode="lines", name="Erro Reconstrução"))
fig.update_layout(title="Erro de Reconstrução com Autoencoder", xaxis_title="Tempo", yaxis_title="Erro")
fig.show()

# Detect anomalies

In [ ]:
start_date_test = pd.to_datetime('2023-12-28 00:00:00')
end_date_test = pd.to_datetime('2024-01-01 00:00:00')

mask = (df_dataset[timestamp] >= start_date_test) & (df_dataset[timestamp] <= end_date_test)
df_anom = df_dataset.loc[mask]
eixo_anom = df_anom[timestamp].to_list()
df_anom = df_anom.drop(columns=[timestamp])

X_anom = scaler.transform(df_anom)

# Projeção no espaço latente
X_anom_latent = encoder.predict(X_anom)

# Reconstrução
X_anom_reconstructed = autoencoder.predict(X_anom)


def get_top_error_variables(original_data, reconstructed_data, feature_names, index, top_n=10):
    """
    Retorna as variáveis com maior erro de reconstrução para uma linha específica.

    Parâmetros:
    - original_data: array com os dados originais escalados
    - reconstructed_data: array com os dados reconstruídos pelo autoencoder
    - feature_names: lista com o nome das variáveis
    - index: índice da linha (int) que será avaliada
    - top_n: número de variáveis com maior erro a retornar

    Retorna:
    - DataFrame com variáveis ordenadas pelo erro de reconstrução
    """
    erro_linha = np.abs(original_data[index] - reconstructed_data[index])

    df_erro = pd.DataFrame({
        'variavel': feature_names,
        'erro_abs': erro_linha
    }).sort_values(by='erro_abs', ascending=False)

    return df_erro.head(top_n)

# Obter variáveis com maior erro para a linha 10 (exemplo de anomalia)
top_vars = get_top_error_variables(X_anom, X_anom_reconstructed, cols, index=10, top_n=10)

print(top_vars)


In [ ]:
# import shap

# # Um modelo wrapper para explicar a reconstrução do autoencoder
# explainer = shap.Explainer(autoencoder.predict, X_anom)

# # Calcula valores SHAP
# shap_values = explainer(X_anom)

# # Visualização
# shap.plots.beeswarm(shap_values)
